In [ ]:
# Clone the repository
# !git clone https://github.com/jonasricker/aeroblade.git

# Navigate into the cloned directory
%cd aeroblade

# Google colab version

In [ ]:
# Install a specific Python version using a more stable method for Colab
# Note: This will restart the kernel.
!pip install -q virtualenv
!virtualenv -p python3.10 aeroblade_env
!source aeroblade_env/bin/activate

c:\PersonalStuff\Monash\Sem3\FIT5230MaliciousAI\Assignments\cloned_repos\aeroblade


c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# Install the core and pip-managed packages.
# We are now pinning a compatible version of peft.
!pip install -v --no-cache-dir --no-input \
    --extra-index-url https://download.pytorch.org/whl/cu128 \
    torch==2.8.0+cu128 torchvision==0.23.0+cu128 \
    peft==0.7.0 \
    scipy numpy \
    scikit-learn scikit-image \
    seaborn tqdm \
    networkx pyarrow \
    diffusers==0.25.1 huggingface-hub==0.25.2 \
    requests PyYAML filelock \
    pillow opencv-python \
    jupyter-client ipykernel ipython jupyter-core \
    clip-interrogator==0.6.0 joblib==1.5.2 lpips==0.1.4 pyiqa==0.1.14.1
    
# Install the project in editable mode
!pip install -e .

# Locally run version. You may need to run this as administrator.

In [ ]:
# import sys
# import subprocess
# import shutil

# # Find the full path to the conda executable
# conda_path = shutil.which("conda")
# if conda_path is None:
#     print("Error: conda executable not found. Please ensure conda is installed and in your system PATH.")
#     # You might want to exit the script here.
# else:
#     print(f"Found conda at: {conda_path}")

#     # List of conda-managed packages
#     conda_packages = [
#         'python=3.10',
#         'numpy=1.26.3',
#         'pandas=2.1.4',
#         'scipy=1.12.0',
#         'matplotlib=3.8.2',
#         'scikit-learn=1.4.0',
#         'scikit-image=0.22.0',
#         'seaborn=0.13.2',
#         'tqdm=4.66.1',
#         'networkx=3.2.1',
#         'pyarrow=15.0.0',
#         'pip'
#     ]

#     # Install conda packages
#     print("Installing conda packages...")
#     try:
#         subprocess.run([conda_path, 'install', '-y', '-c', 'conda-forge'] + conda_packages, check=True)
#         print("Successfully installed conda packages.")
#     except subprocess.CalledProcessError as e:
#         print(f"Failed to install conda packages: {e}")

Found conda at: C:\Users\adria\miniconda3\condabin\conda.BAT
Installing conda packages...


In [ ]:
# import sys
# import subprocess

# # List of pip-managed packages
# pip_packages = [
#     '--extra-index-url', 'https://download.pytorch.org/whl/cu128',
#     'torch==2.8.0+cu128',
#     'torchvision==0.23.0+cu128',
#     'diffusers==0.25.1',
#     'huggingface-hub==0.25.2',
#     'transformers==4.37.2',
#     'requests==2.31.0',
#     'PyYAML==6.0.1',
#     'filelock==3.13.1',
#     'pillow==10.2.0',
#     'opencv-python==4.9.0.80',
#     'jupyter-client==8.6.0',
#     'ipykernel==6.28.0',
#     'ipython==8.20.0',
#     'jupyter-core==5.7.1',
#     'clip-interrogator==0.6.0',
#     'joblib==1.5.2',
#     'lpips==0.1.4',
#     'pyiqa==0.1.14.1'
# ]

# # Install pip packages
# print("Installing pip packages...")
# try:
#     subprocess.run([sys.executable, '-m', 'pip', 'install', '--no-cache-dir', '--no-input'] + pip_packages, check=True)
#     print("Successfully installed pip packages.")
# except subprocess.CalledProcessError as e:
#     print(f"Failed to install pip packages: {e}")

# # Replicates !pip install -e . Might need administrator access.
# print("Installing in editable mode...")
# try:
#     subprocess.run([sys.executable, '-m', 'pip', 'install', '-e', '.'], check=True)
#     print("Successfully installed in editable mode.")
# except subprocess.CalledProcessError as e:
#     print(f"Failed to install in editable mode: {e}")

Installing pip packages...
Installing in editable mode...


# Run the scripts. 
This is the same whether on colab or locally run.
Input your adversarial images as png images into example_images, or specify the folder directory manually.

In [ ]:
### SET IMAGE DIRECTORY HERE
IMG_DIR = "generated/fake_images_sample"

In [ ]:
# Preprocess images with vaeround - increases robustness to generated images
import os
from PIL import Image

VAEROUND_DIR = f"{IMG_DIR}/vaeround"

def list_images(input_dir):
    exts = ('.jpg', '.jpeg', '.png', '.webp', '.bmp', '.tiff')
    for root, _, files in os.walk(input_dir):
        for f in files:
            if f.lower().endswith(exts):
                yield os.path.join(root, f)

def save_img(img: Image.Image, src_path, out_root, transform_name, input_root):
    rel = os.path.relpath(src_path, start=input_root)   # key fix
    dest_dir = os.path.join(out_root, transform_name, os.path.dirname(rel))
    os.makedirs(dest_dir, exist_ok=True)
    dest_path = os.path.join(dest_dir, os.path.basename(rel))
    img.save(dest_path, quality=95)
    return dest_path


def vae_roundtrip_pil(img: Image.Image, vae=None, device="cpu"):
    """
    Convert PIL image -> latent via SD VAE encoder -> decode back.
    vae should be an AutoencoderKL or compatible model from diffusers.
    Works at 1024 resolution expected; will resize image to 1024x1024.
    """
    if vae is None:
        raise RuntimeError("VAE model not provided")
    import torch
    from torchvision import transforms
    img = img.convert("RGB")
    img_resized = img.resize((1024, 1024), resample=Image.BICUBIC)
    to_tensor = transforms.ToTensor()
    x = to_tensor(img_resized).unsqueeze(0).to(device) * 2.0 - 1.0  # [-1,1]
    with torch.no_grad():
        latent = vae.encode(x).latent_dist.sample() * vae.scaling_factor
        decoded = vae.decode(latent / vae.scaling_factor).sample
    decoded = (decoded / 2 + 0.5).clamp(0, 1)
    decoded = (decoded[0].cpu().permute(1, 2, 0).numpy() * 255).astype('uint8')
    return Image.fromarray(decoded)

inp = IMG_DIR
out = IMG_DIR

vae = None
try:
    from diffusers import AutoencoderKL
    # choose a VAE checkpoint compatible with SD. Adjust repo/model as needed.
    vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to("cuda")
    vae.eval()
    print("Loaded VAE.")
except Exception as e:
    print("Failed to load VAE (skipping). Exception:", e)
    vae = None

for path in list_images(inp):
    try:
        img = Image.open(path).convert('RGB')
    except Exception as e:
        print("skip", path, e)
        continue
    
    # VAE roundtrip    
    try:
        vr = vae_roundtrip_pil(img, vae=vae, device="cuda")
        save_img(vr, path, out, "vaeround", inp)
    except Exception as e:
        print("vae roundtrip failed for", path, e)

print("Done. Preprocessed folders in:", out)

c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded VAE.


C:\Users\adria\AppData\Local\Temp\ipykernel_21924\3783955311.py:39: FutureWarning: Accessing config attribute `scaling_factor` directly via 'AutoencoderKL' object attribute is deprecated. Please access 'scaling_factor' over 'AutoencoderKL's config object instead, e.g. 'unet.config.scaling_factor'.
  latent = vae.encode(x).latent_dist.sample() * vae.scaling_factor
C:\Users\adria\AppData\Local\Temp\ipykernel_21924\3783955311.py:40: FutureWarning: Accessing config attribute `scaling_factor` directly via 'AutoencoderKL' object attribute is deprecated. Please access 'scaling_factor' over 'AutoencoderKL's config object instead, e.g. 'unet.config.scaling_factor'.
  decoded = vae.decode(latent / vae.scaling_factor).sample


Done. Preprocessed folders in: real_img/real_nonsqface_sample


In [ ]:
# Image directory specified above
%run scripts/run_aeroblade.py --files-or-dirs {VAEROUND_DIR}


Reconstructing with CompVis/stable-diffusion-v1-1.:   0%|          | 0/30 [00:00<?, ?it/s]
PROGRESS (compute_distances):   0%|          | 0/3 [07:25<?, ?it/s]


AttributeError: module '__main__' has no attribute '__spec__'

In [32]:
import pandas as pd
import os
from glob import glob

# ==== CONFIG ====
decision_boundary = -0.008  # <-- Adjust here

input_dir = "aeroblade_output"

# =================

def classify(distance):
    if 0 >= distance >= decision_boundary:
        return "fake"
    elif distance < decision_boundary:
        return "true"
    else:
        return "unknown"

def extract_label(path, filename):
    path_str = f"{path} {filename}".lower()
    if "real" in path_str:
        return "true"
    elif "fake" in path_str:
        return "fake"
    else:
        return "unknown"

# Find all CSV files starting with "distances"
csv_files = sorted(glob(os.path.join(input_dir, "distances*.csv")))
if not csv_files:
    print(f"No files found starting with 'distances' in {input_dir}")
    exit()

all_results = []

for csv_file in csv_files:
    print(f"\n=== Processing {os.path.basename(csv_file)} ===")
    df = pd.read_csv(csv_file)
    df_max = df[df["repo_id"] == "max"].copy()

    if df_max.empty:
        print("No 'max' rows found, skipping.")
        continue

    df_max["prediction"] = df_max["distance"].apply(classify)
    df_max["ground_truth"] = df_max.apply(
        lambda row: extract_label(row["dir"], row["file"]), axis=1
    )

    # Prediction counts (all images)
    counts = df_max["prediction"].value_counts(dropna=False)
    print("\nPrediction counts:")
    print(counts.to_string())

    # Evaluate where ground truth is known
    valid = df_max[df_max["ground_truth"] != "unknown"]
    if not valid.empty:
        accuracy = (valid["prediction"] == valid["ground_truth"]).mean()
        print(f"\nAccuracy (on {len(valid)} images with ground truth): {accuracy:.4f}")

        confusion = pd.crosstab(valid["ground_truth"], valid["prediction"],
                                rownames=["Actual"], colnames=["Predicted"], dropna=False)
        print("\nConfusion Matrix:")
        print(confusion)
    else:
        accuracy = None
        confusion = None
        print("⚠️ No ground-truth labels found in this file.")

    # Keep for overall tally
    df_max["source_file"] = os.path.basename(csv_file)
    all_results.append(df_max)

# ==== Aggregate overall stats ====
if all_results:
    df_all = pd.concat(all_results, ignore_index=True)

    print("\n==============================")
    print("=== Overall Summary Across All Files ===")

    # Prediction counts
    total_counts = df_all["prediction"].value_counts(dropna=False)
    print("\nTotal prediction counts (all images):")
    print(total_counts.to_string())

    # Accuracy + confusion for known labels
    valid_all = df_all[df_all["ground_truth"] != "unknown"]
    if not valid_all.empty:
        overall_accuracy = (valid_all["prediction"] == valid_all["ground_truth"]).mean()
        print(f"\nOverall Accuracy (on {len(valid_all)} labeled images): {overall_accuracy:.4f}")

        overall_confusion = pd.crosstab(valid_all["ground_truth"], valid_all["prediction"],
                                        rownames=["Actual"], colnames=["Predicted"], dropna=False)
        print("\nOverall Confusion Matrix:")
        print(overall_confusion)
    else:
        print("\n⚠️ No ground-truth labels found across any files.")
else:
    print("\nNo valid 'max' data found in any distances CSV.")



=== Processing distances_fake_image_sample.csv ===

Prediction counts:
prediction
fake    455
true     35

Accuracy (on 490 images with ground truth): 0.9286

Confusion Matrix:
Predicted  fake  true
Actual               
fake        455    35

=== Processing distances_fakegen_vaeround.csv ===

Prediction counts:
prediction
fake    15

Accuracy (on 15 images with ground truth): 1.0000

Confusion Matrix:
Predicted  fake
Actual         
fake         15

=== Processing distances_real_vae_1024.csv ===

Prediction counts:
prediction
true    428
fake     72

Accuracy (on 500 images with ground truth): 0.8560

Confusion Matrix:
Predicted  fake  true
Actual               
true         72   428

=== Overall Summary Across All Files ===

Total prediction counts (all images):
prediction
fake    542
true    463

Overall Accuracy (on 1005 labeled images): 0.8935

Overall Confusion Matrix:
Predicted  fake  true
Actual               
fake        470    35
true         72   428
